# Titulo

# MVP – Modelo de Clasificación de Clientes para Suscripción a Depósitos

## Objetivo del notebook:
Construir un modelo predictivo que permita anticipar si un cliente aceptará una campaña de marketing de suscripción a un depósito bancario, y generar los archivos necesarios para alimentar un dashboard de seguimiento.

#Librerias

In [1]:
!pip install scikit-learn
!pip install collections
!pip install imblearn.over_sampling
!pip install streamlit

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement imblearn.over_sampling (from versions: none)
ERROR: No matching distribution found for imblearn.over_sampling
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import files
from scipy.stats.mstats import winsorize
from scipy.stats import gaussian_kde, randint, uniform
from sklearn.metrics import (accuracy_score, f1_score, precision_score, roc_curve,
                             recall_score, confusion_matrix, ConfusionMatrixDisplay,
                             classification_report, roc_auc_score, precision_recall_curve)
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from google.colab import drive
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
import datetime
import os

# dataset

1. Carga y exploración del dataset
Lectura de base de datos de clientes.

In [3]:
uploaded = files.upload()
filename = next(iter(uploaded))
data = pd.read_csv(filename, sep=';')


Saving bank-additional-full.csv to bank-additional-full.csv


# Feature engineering

Se agregaron las columnas

1.   contactosTotales para ver cuantas veces se lo contacto previamente
2.   fueContactado para ver si antes se lo habia contactado
3. campañasExitosasprevias
4. segmento que divide los clientes segun socioeconomia (es media basica la division)
5. mesNumero pase los meses a numeros 1=enero; 2=febrero y asi
6. hay algunas que tienen log, log2, winz_log, bin que son las que modifique en base al skew

## Feature construction

In [4]:
# Total de contactos
data['contactosTotales'] = data['campaign'] + data['previous']


# Campañas exitosas previas
data['campañasExitosasPrevias'] = ((data['poutcome'] == 'success') & (data['previous'] > 0)).astype(int)
data["campañasExitosasPrevias"].value_counts()

,count
campañasExitosasPrevias,
0,39815
1,1373


## Encoding

In [6]:
encode = data.copy()
encode.drop(columns=['default',"poutcome"], inplace=True)
encode.dtypes.to_frame()

,0
age,int64
job,object
marital,object
education,object
housing,object
loan,object
contact,object
month,object
day_of_week,object
duration,int64


In [7]:
encode.sample(5)

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,...,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,contactosTotales,campañasExitosasPrevias
25524,51,housemaid,married,high.school,no,no,cellular,nov,wed,355,...,999,0,-0.1,93.200,-42.0,4.120,5195.8,no,2,0
8864,40,entrepreneur,married,basic.6y,no,no,telephone,jun,thu,85,...,999,0,1.4,94.465,-41.8,4.866,5228.1,no,2,0
18413,45,admin.,married,university.degree,no,no,cellular,jul,thu,196,...,999,0,1.4,93.918,-42.7,4.968,5228.1,no,1,0
18728,54,admin.,married,university.degree,yes,no,cellular,jul,thu,65,...,999,0,1.4,93.918,-42.7,4.968,5228.1,no,43,0
38221,63,retired,married,high.school,no,no,cellular,oct,wed,335,...,999,0,-3.4,92.431,-26.9,0.740,5017.5,yes,1,0


### target

In [45]:
encode['y'] = encode['y'].astype(str)
encode['y'] = encode['y'].map({'no': 0, 'yes': 1})

### occupation

In [8]:
categorical_cols = encode.select_dtypes(include=['object']).columns
cardinality = {}
for col in categorical_cols:
    cardinality[col] = encode[col].nunique()


for col, card in cardinality.items():
    print(f"{col}: {card}")


job: 12
marital: 4
education: 8
housing: 3
loan: 3
contact: 2
month: 10
day_of_week: 5
y: 2


In [9]:
encode["previous"].value_counts()

,count
previous,
0,35563
1,4561
2,754
3,216
4,70
5,18
6,5
7,1


### Job

In [46]:
job_target_mean = encode.groupby('job')['y'].mean()
encode['job_target_mean'] = encode['job'].map(job_target_mean)

encode.drop('job', axis=1, inplace=True)
encode.head()

,age,month,day_of_week,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,previous_bin,job_target_mean
0,56,5,1,261,1,999,0,1.1,93.994,-36.4,...,1,0,0,1,0,0,0,1,0,0.100000
1,57,5,1,149,1,999,0,1.1,93.994,-36.4,...,1,0,0,1,0,0,0,1,0,0.081381
2,37,5,1,226,1,999,0,1.1,93.994,-36.4,...,0,0,1,1,0,0,0,1,0,0.081381
3,40,5,1,151,1,999,0,1.1,93.994,-36.4,...,1,0,0,1,0,0,0,1,0,0.129726
4,56,5,1,307,1,999,0,1.1,93.994,-36.4,...,1,0,0,0,0,1,0,1,0,0.081381


### Marital

In [14]:
marital_dummies = pd.get_dummies(encode['marital'], prefix='marital', dtype=int)
encode = pd.concat([encode, marital_dummies], axis=1)
encode.drop('marital', axis=1, inplace=True)
encode.head()

,age,job,education,housing,loan,contact,month,day_of_week,duration,campaign,...,cons.conf.idx,euribor3m,nr.employed,y,contactosTotales,campañasExitosasPrevias,marital_divorced,marital_married,marital_single,marital_unknown
0,56,housemaid,basic.4y,no,no,telephone,may,mon,261,1,...,-36.4,4.857,5191.0,no,1,0,0,1,0,0
1,57,services,high.school,no,no,telephone,may,mon,149,1,...,-36.4,4.857,5191.0,no,1,0,0,1,0,0
2,37,services,high.school,yes,no,telephone,may,mon,226,1,...,-36.4,4.857,5191.0,no,1,0,0,1,0,0
3,40,admin.,basic.6y,no,no,telephone,may,mon,151,1,...,-36.4,4.857,5191.0,no,1,0,0,1,0,0
4,56,services,high.school,no,yes,telephone,may,mon,307,1,...,-36.4,4.857,5191.0,no,1,0,0,1,0,0


### Education

In [15]:
education_counts = encode['education'].value_counts(normalize=True)
encode['education_freq_encode'] = encode['education'].map(education_counts)
encode.drop('education', axis=1, inplace=True)
encode.head()

,age,job,housing,loan,contact,month,day_of_week,duration,campaign,pdays,...,euribor3m,nr.employed,y,contactosTotales,campañasExitosasPrevias,marital_divorced,marital_married,marital_single,marital_unknown,education_freq_encode
0,56,housemaid,no,no,telephone,may,mon,261,1,999,...,4.857,5191.0,no,1,0,0,1,0,0,0.101389
1,57,services,no,no,telephone,may,mon,149,1,999,...,4.857,5191.0,no,1,0,0,1,0,0,0.231014
2,37,services,yes,no,telephone,may,mon,226,1,999,...,4.857,5191.0,no,1,0,0,1,0,0,0.231014
3,40,admin.,no,no,telephone,may,mon,151,1,999,...,4.857,5191.0,no,1,0,0,1,0,0,0.055647
4,56,services,no,yes,telephone,may,mon,307,1,999,...,4.857,5191.0,no,1,0,0,1,0,0,0.231014


### Housing, Loan y Contact

In [16]:
housing_dummies = pd.get_dummies(encode['housing'], prefix='housing', dtype=int)
loan_dummies = pd.get_dummies(encode['loan'], prefix='loan', dtype=int)
contact_dummies = pd.get_dummies(encode['contact'], prefix='contact', dtype=int)

encode = pd.concat([encode, housing_dummies, loan_dummies, contact_dummies], axis=1)

encode.drop(['housing', 'loan', 'contact'], axis=1, inplace=True)

encode.head()

,age,job,month,day_of_week,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,...,marital_unknown,education_freq_encode,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone
0,56,housemaid,may,mon,261,1,999,0,1.1,93.994,...,0,0.101389,1,0,0,1,0,0,0,1
1,57,services,may,mon,149,1,999,0,1.1,93.994,...,0,0.231014,1,0,0,1,0,0,0,1
2,37,services,may,mon,226,1,999,0,1.1,93.994,...,0,0.231014,0,0,1,1,0,0,0,1
3,40,admin.,may,mon,151,1,999,0,1.1,93.994,...,0,0.055647,1,0,0,1,0,0,0,1
4,56,services,may,mon,307,1,999,0,1.1,93.994,...,0,0.231014,1,0,0,0,0,1,0,1


### Month y Day of Week

In [17]:
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
encode['month'] = encode['month'].map(month_mapping)


day_mapping = {'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5}
encode['day_of_week'] = encode['day_of_week'].map(day_mapping)

## Discretizacion

### previous

In [18]:
#Binning de previous
encode['previous_bin'] = pd.cut(data['previous'], bins=[-1, 0, 2, np.inf], labels=[0, 1, 2])
encode["previous_bin"] = encode["previous_bin"].astype(int)
encode["previous_bin"].value_counts()
# 0: Nada (0 contactos)
# 1: Poco (1 o 2 contactos)
# 2: Mucho (3-7)

,count
previous_bin,
0,35563
1,5315
2,310


# aplicar tecnicas de oversampleo

2. Definición del problema
Clasificación binaria (y: "yes"/"no").

Evidencia de desbalanceo de clases (alta proporción de “no”).

### SMOTE

In [48]:
x = encode
smote_df = x.copy()
smote_df_y = smote_df['y']
smote_df_x = smote_df.drop('y', axis=1)

In [49]:
smote = SMOTE(sampling_strategy = 0.3, random_state=42)
X_resampled, y_resampled = smote.fit_resample(smote_df_x, smote_df_y)

In [50]:
X = X_resampled.copy()
y = y_resampled.copy()

# Modelo puesto en producción

3. Preparación del modelo
* División 80/20 del dataset.

* Entrenamiento de un modelo XGBClassifier.

* Predicción de clases y probabilidades.

4. Cálculo de métricas
* precision, recall, f1, roc_auc, classification_report.

* Generación de curvas ROC y PR.

In [74]:
project_path = "/content/gdrive/MyDrive/Detección_clientes_de_banco/models/Resultados"
os.makedirs(project_path, exist_ok=True)  # Crea si no existe

In [75]:
def modelo(x, y, column_names=None):
    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y
    )
    xgb_model = XGBClassifier(
        n_estimators=100, learning_rate=0.1, max_depth=6,
        eval_metric='logloss', random_state=42
    )
    xgb_model.fit(X_train, y_train)

    y_pred = xgb_model.predict(X_test)
    y_proba = xgb_model.predict_proba(X_test)[:, 1]

    # Métricas
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    metrics_df = pd.DataFrame([{
        'date': pd.Timestamp.now(),
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc
    }])
    metrics_df.to_csv('metrics.csv', mode='a',
                      header=not pd.io.common.file_exists('metrics.csv'),
                      index=False)

    fpr, tpr, _ = roc_curve(y_test, y_proba)
    pr, rc, _ = precision_recall_curve(y_test, y_proba)
    pd.DataFrame({'fpr': fpr, 'tpr': tpr}).to_csv("roc_curve.csv", index=False)
    pd.DataFrame({'precision_curve': pr, 'recall_curve': rc}).to_csv("pr_curve.csv", index=False)

    df_resultados = pd.DataFrame({
        'Probabilidad_clase_1': y_proba,
        'Predicción': y_pred,
        'Real': y_test.values
    })
    df_resultados.to_csv('predicciones.csv', index=False)

    return df_resultados, metrics_df, y_proba, X_test, y_test, y_pred



## Evaluar modelo

In [76]:
# EJECUTAR
resultados, metrics_df, y_proba, X_test, y_test, y_pred = modelo(X, y)

## Otras métricas

In [77]:
df_segmentos = pd.DataFrame(X_test, columns=[f'PC{i+1}' for i in range(X_test.shape[1])])
df_segmentos['probabilidad_clase_1'] = y_proba
df_segmentos['real'] = y_test.values
df_segmentos['prediccion'] = y_pred
df_segmentos.to_csv('clientes_segmentados.csv', index=False)

In [78]:
#Necesario hablar con departamento de costos y marketing para determinar los valores de costos e ingresos por llamadas
costo_llamada = 1.0                # costo estimado por llamada
ingreso_por_cliente = 100          # estimación de ingresos por cliente aceptado

In [79]:
# Escenario 1: antes del modelo
llamadas_total = len(y_test)
tasa_aceptacion_hist = sum(y_test) / llamadas_total
roi_hist = (sum(y_test) * ingreso_por_cliente - llamadas_total * costo_llamada) / (llamadas_total * costo_llamada)

# Escenario 2: después del modelo
# Por ejemplo, suponiendo que solo contactas a los que superan cierto umbral de probabilidad
threshold = 0.5
contactados_modelo = sum(y_proba >= threshold)
aceptadas_modelo = sum((y_proba >= threshold) & (y_test == 1))  # solo los que respondieron que sí entre los contactados
roi_modelo = (aceptadas_modelo * ingreso_por_cliente - contactados_modelo * costo_llamada) / (contactados_modelo * costo_llamada)

# Unimos los dos escenarios
impact_df = pd.DataFrame([
    {'date': datetime.datetime.now(),
     'scenario': 'before_model',
     'llamadas': llamadas_total,
     'aceptadas': int(sum(y_test)),
     'roi_estimado': roi_hist},
    {'date': datetime.datetime.now(),
     'scenario': 'after_model',
     'llamadas': contactados_modelo,
     'aceptadas': int(aceptadas_modelo),
     'roi_estimado': roi_modelo}
])
# Append al CSV
impact_df.to_csv('impact.csv', mode='a', header=not pd.io.common.file_exists('impact.csv'), index=False)


In [80]:
training_status = pd.DataFrame([{'last_training_date': datetime.datetime.now(),
               'n_samples': len(X),
               'drift_metric': None  # Por ahora None; aquí podrías calcular PSI u otras métricas
              }])
training_status.to_csv('training_status.csv', index=False)

# Exportación

5. Exportación de resultados
metrics.csv: métricas por fecha.

predicciones.csv: predicciones vs reales.

clientes_segmentados.csv: base enriquecida por cliente.

roc_curve.csv, pr_curve.csv: curvas.

impact.csv: comparación antes/después (ROI).

training_status.csv: fecha del último entrenamiento.


(TODOS LOS RESULTADOS DEBEN ENCONTRARSE EN LA CARPETA "models". Si hay algun archivo que NO se encuentre en la carpeta, descargarla y subirla de inmediato)

In [81]:
metrics_df.to_csv(f"{project_path}/metrics.csv", index=False)
df_segmentos.to_csv(f"{project_path}/clientes_segmentados.csv", index=False)
impact_df.to_csv(f"{project_path}/impact.csv", index=False)
training_status.to_csv(f"{project_path}/training_status.csv", index=False)
resultados.to_csv(f"{project_path}/predicciones.csv", index=False)

## Supuestos del modelo
El dataset ya tiene las variables necesarias para modelar (age, job, balance, etc.).

Se calcula ROI bajo hipótesis de costo/cliente e ingreso estimado. (Se requiere contactar con el departamento de marketing para conocer los costos y beneficios de las campañas de llamadas)

Se asume que job_target_mean está disponible como feature.


## Resultado final:
Se exportan todos los CSV necesarios para el dashboard interactivo.

El modelo predice correctamente a partir de variables crudas, como prueba de concepto (PoC).

Listo para integrarse en un flujo automatizado.